In [1]:
from transformers import (
    AutoTokenizer, 
    AutoModelForTokenClassification, 
    Trainer, 
    TrainingArguments, 
    EarlyStoppingCallback,
    TrainerCallback,
    PreTrainedTokenizerBase
)
import numpy as np
import os
import torch
from torch import nn
from sklearn.model_selection import train_test_split
from transformers import EvalPrediction, AutoConfig
import json
from datetime import datetime
from typing import List, Dict, Tuple, Optional, Union
from dataclasses import dataclass
from torch.utils.data import Dataset
from sklearn.metrics import precision_recall_fscore_support, accuracy_score #confusion_matrix
import json
os.environ["TOKENIZERS_PARALLELISM"] = "true"

/opt/miniconda3/envs/checkthat/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
@dataclass
class DataCollatorForDynamicTokenClassification:
    """
    Data collator that will dynamically pad the inputs received.
    """
    tokenizer: PreTrainedTokenizerBase
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    label_pad_token_id: int = -100

    def __call__(self, features: List[Dict[str, torch.Tensor]]) -> Dict[str, torch.Tensor]:
        # Get sequence lengths and find max length in batch
        batch_size = len(features)
        if self.pad_to_multiple_of is not None:
            max_length = max(len(x["input_ids"]) for x in features)
            if max_length % self.pad_to_multiple_of != 0:
                max_length = ((max_length // self.pad_to_multiple_of) + 1) * self.pad_to_multiple_of
        else:
            max_length = max(len(x["input_ids"]) for x in features)

        # Initialize padded tensors
        input_ids = torch.full((batch_size, max_length), self.tokenizer.pad_token_id, dtype=torch.long)
        attention_mask = torch.zeros((batch_size, max_length), dtype=torch.long)
        labels = torch.full((batch_size, max_length), self.label_pad_token_id, dtype=torch.long)

        # Fill tensors with actual values
        for i, feature in enumerate(features):
            seq_length = len(feature["input_ids"])
            input_ids[i, :seq_length] = feature["input_ids"]
            attention_mask[i, :seq_length] = feature["attention_mask"]
            labels[i, :seq_length] = feature["labels"]

        return {
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "labels": labels,
        }

class TokenClassificationDataset(Dataset):
    def __init__(self, tokenized_inputs: Dict[str, List], labels: List[List[int]], pad_token_id: int):
        self.encodings = tokenized_inputs
        self.labels = labels
        self.pad_token_id = pad_token_id

    def __getitem__(self, idx: int) -> Dict[str, torch.Tensor]:
        # Remove padding from the encodings and labels
        item = {}
        for key, val in self.encodings.items():
            if key == "input_ids":
                # Find the last non-padding token
                last_token = len(val[idx]) - 1
                while last_token > 0 and val[idx][last_token] == self.pad_token_id:
                    last_token -= 1
                item[key] = torch.tensor(val[idx][:last_token + 1])
            elif key == "attention_mask":
                item[key] = torch.tensor(val[idx][:len(item["input_ids"])])
        
        # Trim labels to match input length
        item["labels"] = torch.tensor(self.labels[idx][:len(item["input_ids"])])
        return item

    def __len__(self) -> int:
        return len(self.labels)
    
class WeightedTokenClassification(AutoModelForTokenClassification):
    def __init__(self, config):
        super().__init__(config)
        self.focal_loss_gamma = 4.0  # Increased focal loss parameter
        self.class_weights = None
        self.threshold = 0.4  # Threshold for O class prediction
        
    def compute_class_weights(self, labels):
        # Remove padding tokens
        valid_labels = labels[labels != -100]
        # Count class frequencies
        unique, counts = np.unique(valid_labels, return_counts=True)
        total = np.sum(counts)
        
        # Extremely aggressive weighting scheme
        weights = np.zeros(len(self.config.id2label))
        weights[unique] = total / (len(unique) * counts)
        
        # Nearly eliminate 'O' class weight and heavily boost propaganda classes
        weights[0] = weights[0] * 0.00001  # Extremely aggressive downweighting of 'O' class
        weights[1:] = weights[1:] * 100.0   # Much stronger boost for propaganda classes
        
        # Apply cubic root to maintain some differences while reducing extreme values
        weights = np.cbrt(weights)
        
        # Normalize weights
        weights = weights / np.mean(weights[weights != 0])
        
        return torch.FloatTensor(weights).to(labels.device)

    def focal_loss(self, logits, labels, class_weights):
        """
        Compute focal loss with higher gamma for more focus on hard examples
        """
        ce_loss = torch.nn.functional.cross_entropy(
            logits, 
            labels, 
            weight=class_weights,
            ignore_index=-100,
            reduction='none'
        )
        
        pt = torch.exp(-ce_loss)
        focal_loss = (1 - pt) ** self.focal_loss_gamma * ce_loss
        return focal_loss.mean()

    def forward(
        self,
        input_ids: Optional[torch.Tensor] = None,
        attention_mask: Optional[torch.Tensor] = None,
        labels: Optional[torch.Tensor] = None,
        **kwargs
    ) -> Tuple[torch.Tensor, ...]:
        outputs = super().forward(
            input_ids=input_ids,
            attention_mask=attention_mask,
            **kwargs
        )
        
        if labels is not None:
            class_weights = self.compute_class_weights(labels.cpu().numpy())
            logits = outputs.logits
            
            if self.training:
                # Custom logits adjustment to discourage 'O' class predictions
                logits[:, :, 0] = logits[:, :, 0] - 2.0  # Penalize 'O' class logits
                
                # Create smoothed labels with propaganda class boosting
                smoothed_labels = torch.zeros_like(logits)
                valid_mask = labels != -100
                
                # One-hot encoding
                smoothed_labels[valid_mask] = torch.nn.functional.one_hot(
                    labels[valid_mask], 
                    num_classes=self.config.num_labels
                ).float()
                
                # Stronger label smoothing for propaganda classes
                propaganda_mask = (labels != -100) & (labels != 0)
                smoothed_labels[propaganda_mask] = (
                    smoothed_labels[propaganda_mask] * 0.95 +  # Higher confidence in main class
                    0.05 / (self.config.num_labels - 1)       # Less distribution to other classes
                )
                
                # Add auxiliary loss to encourage propaganda prediction
                probs = torch.softmax(logits, dim=-1)
                o_class_penalty = torch.mean(probs[:, :, 0] * (labels != -100).float())
                
                # Compute primary loss with smoothed labels
                main_loss = -torch.sum(
                    smoothed_labels * torch.log_softmax(logits, dim=-1),
                    dim=-1
                )
                main_loss = (main_loss * (labels != -100).float()).mean()
                
                # Combine losses
                loss = main_loss + 0.5 * o_class_penalty
                
            else:
                # Use focal loss for evaluation
                loss = self.focal_loss(
                    logits.view(-1, self.config.num_labels),
                    labels.view(-1),
                    class_weights
                )
            
            outputs.loss = loss
            
        return outputs

# Modified training setup
def setup_model(model_name: str, num_labels: int):
    config = AutoConfig.from_pretrained(
        model_name, 
        num_labels=num_labels,
        hidden_dropout_prob=0.3,    # Increased dropout
        attention_probs_dropout_prob=0.3
    )
    model = WeightedTokenClassification.from_pretrained(
        model_name,
        config=config
    )
    return model

class MetricsCallback(TrainerCallback):
    """
    Callback to save detailed metrics after each evaluation.
    """
    def __init__(self, output_dir: str, id_to_label: Dict[int, str]):
        self.output_dir = output_dir
        self.id_to_label = id_to_label
        self.best_metrics = None
        self.best_step = None
        
    def on_evaluate(self, args, state, control, metrics, **kwargs):
        """Called after each evaluation."""
        # Create epoch-specific directory
        epoch_dir = os.path.join(self.output_dir, f"epoch_{state.epoch}")
        os.makedirs(epoch_dir, exist_ok=True)
        
        # Save detailed metrics for this epoch
        detailed_metrics = {
            "epoch": state.epoch,
            "step": state.global_step,
            "overall_propaganda_metrics": {
                "precision": metrics.get("eval_propaganda_precision", 0),
                "recall": metrics.get("eval_propaganda_recall", 0),
                "f1": metrics.get("eval_propaganda_f1", 0)
            },
            "per_class_metrics": {}
        }
        
        # Add per-class metrics
        for i in range(len(self.id_to_label)):
            class_name = self.id_to_label[i]
            detailed_metrics["per_class_metrics"][class_name] = {
                "precision": metrics["eval_per_class_precision"][i],
                "recall": metrics["eval_per_class_recall"][i],
                "f1": metrics["eval_per_class_f1"][i],
                "support": metrics["eval_support"][i]
            }
        
        # Track best metrics
        current_f1 = metrics.get("eval_propaganda_f1", 0)
        if self.best_metrics is None or current_f1 > self.best_metrics["overall_propaganda_metrics"]["f1"]:
            self.best_metrics = detailed_metrics
            self.best_step = state.global_step
            
            # Save best metrics separately
            best_metrics_path = os.path.join(self.output_dir, "best_metrics.json")
            with open(best_metrics_path, 'w', encoding='utf8') as f:
                json.dump({
                    "best_step": self.best_step,
                    "metrics": self.best_metrics
                }, f, ensure_ascii=False, indent=2)
        
        # Save epoch metrics
        metrics_path = os.path.join(epoch_dir, 'detailed_metrics.json')
        with open(metrics_path, 'w', encoding='utf8') as f:
            json.dump(detailed_metrics, f, ensure_ascii=False, indent=2)
        
        '''        # Save confusion matrix if available
        if "eval_confusion_matrix" in metrics:
            confusion_path = os.path.join(epoch_dir, 'confusion_matrix.npy')
            np.save(confusion_path, np.array(metrics["eval_confusion_matrix"]))'''
            
        # Print summary of propaganda metrics
        print("\nPropaganda Detection Metrics:")
        print(f"Precision: {metrics.get('eval_propaganda_precision', 0):.4f}")
        print(f"Recall: {metrics.get('eval_propaganda_recall', 0):.4f}")
        print(f"F1: {metrics.get('eval_propaganda_f1', 0):.4f}")
        
        # Print per-class F1 scores for non-O classes
        print("\nPer-class F1 scores (excluding 'O'):")
        for i in range(1, len(self.id_to_label)):  # Skip 'O' class
            class_name = self.id_to_label[i]
            f1_score = metrics["eval_per_class_f1"][i]
            support = metrics["eval_support"][i]
            if support > 0:  # Only show classes that appear in the evaluation set
                print(f"{class_name}: {f1_score:.4f} (support: {support})")
    
    def on_train_end(self, args, state, control, **kwargs):
        """Called at the end of training - print best results."""
        if self.best_metrics is not None:
            print("\nBest Model Performance:")
            print(f"Step: {self.best_step}")
            best_f1 = self.best_metrics["overall_propaganda_metrics"]["f1"]
            print(f"Best Propaganda F1: {best_f1:.4f}")


@dataclass
class TokenClassificationConfig:
    model_name: str = 'microsoft/mdeberta-v3-base'
    max_length: int = 512
    stride: int = 256
    num_labels: int = 24
    output_dir: str = '/home/lgiordano/LUCA/checkthat_GITHUB/models/sliding_window'
    
    # Add training specific parameters
    train_batch_size: int = 16
    eval_batch_size: int = 16
    num_train_epochs: int = 20
    learning_rate: float = 5e-5
    weight_decay: float = 0.01
    warmup_ratio: float = 0.1
    gradient_accumulation_steps: int = 2
    logging_steps: int = 50
    eval_steps: int = 100
    save_total_limit: int = 2
    label_smoothing_factor: float = 0.1

    def __post_init__(self):
        self.date_time = datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
        self.full_output_dir = os.path.join(self.output_dir, self.date_time)
        # Add id_to_label mapping
        self.id_to_label = {
            0: "O", 
            1: "Appeal_to_Authority",
            2: "Appeal_to_Popularity",
            3: "Appeal_to_Values",
            4: "Appeal_to_Fear-Prejudice",
            5: "Flag_Waving",
            6: "Causal_Oversimplification",
            7: "False_Dilemma-No_Choice",
            8: "Consequential_Oversimplification",
            9: "Straw_Man",
            10: "Red_Herring",
            11: "Whataboutism",
            12: "Slogans",
            13: "Appeal_to_Time",
            14: "Conversation_Killer",
            15: "Loaded_Language",
            16: "Repetition",
            17: "Exaggeration-Minimisation",
            18: "Obfuscation-Vagueness-Confusion",
            19: "Name_Calling-Labeling",
            20: "Doubt",
            21: "Guilt_by_Association",
            22: "Appeal_to_Hypocrisy",
            23: "Questioning_the_Reputation"
        }


def encode_tags(tags: List[Dict], 
                token_offsets: List[Tuple[int, int]], 
                label_to_id: Dict[str, int], 
                max_length: int) -> List[int]:
    """Encode tags for token classification."""
    token_labels = ["O"] * len(token_offsets)
    
    for annotation in tags:
        label = annotation["tag"]
        start = annotation["start"]
        end = annotation["end"]
        
        # Find all tokens that overlap with the annotation span
        for idx, (token_start, token_end) in enumerate(token_offsets):
            if token_start < end and token_end > start:
                token_labels[idx] = label
    
    # Convert string labels to ids
    token_labels = [label_to_id[label] for label in token_labels]
    
    # Pad with -100 (ignored in loss calculation)
    if len(token_labels) < max_length:
        token_labels += [-100] * (max_length - len(token_labels))
    
    return token_labels

def preprocess_data(texts: List[str], 
                    annotations: List[Dict], 
                    tokenizer, 
                    config: TokenClassificationConfig,
                    label_to_id: Dict[str, int]) -> Tuple[Dict[str, List], List[List[int]]]:
    """Preprocess texts and annotations into model inputs."""
    input_ids = []
    attention_masks = []
    labels = []

    for text, tags in zip(texts, annotations):
        text_length = len(text)
        start_idx = 0

        while start_idx < text_length:
            # Tokenize text chunk with overlap but without padding
            encoded_chunk = tokenizer(
                text[start_idx:start_idx + config.max_length],
                truncation=True,
                max_length=config.max_length,
                padding=False,  # Don't pad here - we'll do it dynamically
                return_offsets_mapping=True
            )

            input_ids.append(encoded_chunk["input_ids"])
            attention_masks.append(encoded_chunk["attention_mask"])
            token_offsets = encoded_chunk.pop("offset_mapping")
            
            chunk_labels = encode_tags(
                tags, 
                token_offsets, 
                label_to_id, 
                len(encoded_chunk["input_ids"])  # Use actual length instead of max_length
            )
            labels.append(chunk_labels)

            start_idx += config.max_length - config.stride

    return {"input_ids": input_ids, "attention_mask": attention_masks}, labels

def compute_metrics(pred: EvalPrediction) -> Dict[str, float]:
    """
    Compute metrics for token classification, handling class imbalance 
    and reporting per-class metrics.
    """
    labels = pred.label_ids.flatten()
    preds = np.argmax(pred.predictions, axis=2).flatten()
    
    # Filter out padding tokens (-100)
    mask = labels != -100
    labels = labels[mask]
    preds = preds[mask]

    # Calculate metrics excluding the 'O' class for a more realistic evaluation
    non_o_mask = labels != 0  # Assuming 0 is the ID for 'O' class
    propaganda_labels = labels[non_o_mask]
    propaganda_preds = preds[non_o_mask]

    # Calculate metrics only for propaganda classes
    propaganda_precision, propaganda_recall, propaganda_f1, _ = precision_recall_fscore_support(
        propaganda_labels, 
        propaganda_preds, 
        average="micro",
        labels=list(range(1, 24))  # Exclude 'O' class
    )

    # Calculate per-class metrics
    per_class_precision, per_class_recall, per_class_f1, support = precision_recall_fscore_support(
        labels, 
        preds, 
        labels=list(range(24)
        ),  # Include all classes
        zero_division=0
    )

    results = {
        "propaganda_precision": propaganda_precision,
        "propaganda_recall": propaganda_recall,
        "propaganda_f1": propaganda_f1,
        "per_class_precision": per_class_precision.tolist(),
        "per_class_recall": per_class_recall.tolist(),
        "per_class_f1": per_class_f1.tolist(),
        "support": support.tolist()
    }

    '''    # Add confusion matrix
    confusion = confusion_matrix(
        labels, 
        preds, 
        labels=list(range(24))
    )
    results["confusion_matrix"] = confusion.tolist()'''

    return results

def save_detailed_metrics(metrics: Dict[str, float], 
                         output_dir: str,
                         id_to_label: Dict[int, str]):
    """Save detailed metrics with per-class breakdown."""
    os.makedirs(output_dir, exist_ok=True)
    
    # Create detailed report
    detailed_metrics = {
        "overall_propaganda_metrics": {
            "precision": metrics["propaganda_precision"],
            "recall": metrics["propaganda_recall"],
            "f1": metrics["propaganda_f1"]
        },
        "per_class_metrics": {}
    }

    # Add per-class metrics
    for i in range(len(id_to_label)):
        class_name = id_to_label[i]
        detailed_metrics["per_class_metrics"][class_name] = {
            "precision": metrics["per_class_precision"][i],
            "recall": metrics["per_class_recall"][i],
            "f1": metrics["per_class_f1"][i],
            "support": metrics["support"][i]
        }

    # Save metrics
    metrics_path = os.path.join(output_dir, 'detailed_results.json')
    with open(metrics_path, 'w', encoding='utf8') as f:
        json.dump(detailed_metrics, f, ensure_ascii=False, indent=2)

    # Save confusion matrix separately
    #confusion_path = os.path.join(output_dir, 'confusion_matrix.npy')
    #np.save(confusion_path, np.array(metrics["confusion_matrix"]))

class PropagandaThresholdTrainer(Trainer):
    def prediction_step(self, model, inputs, prediction_loss_only, ignore_keys=None):
        has_labels = "labels" in inputs
        inputs = self._prepare_inputs(inputs)

        # Compute predictions and loss
        with torch.no_grad():
            outputs = model(**inputs)
            logits = outputs.logits

            # Apply threshold to 'O' class predictions during evaluation
            if not prediction_loss_only:
                probs = torch.softmax(logits, dim=-1)
                o_class_probs = probs[:, :, 0]
                
                # If 'O' class probability is below threshold, redistribute to other classes
                mask = o_class_probs < 0.9  # High threshold for 'O' class
                probs[mask] = probs[mask] * (1 - o_class_probs[mask].unsqueeze(-1))
                probs[mask] = probs[mask] / probs[mask].sum(dim=-1, keepdim=True)
                
                # Convert back to logits
                logits = torch.log(probs + 1e-10)

        if prediction_loss_only:
            return outputs.loss, None, None

        return outputs.loss, logits, inputs["labels"]


def setup_training(config, train_dataset, val_dataset, model, tokenizer):
    num_training_steps = (
        len(train_dataset) // config.train_batch_size * config.num_train_epochs
    )
    num_warmup_steps = int(num_training_steps * config.warmup_ratio)
    
    # Create custom data collator for dynamic padding
    data_collator = DataCollatorForDynamicTokenClassification(
        tokenizer=tokenizer,
        padding=True,
        max_length=config.max_length,
        pad_to_multiple_of=8  # Optimize for GPU efficiency
    )
    
    training_args = TrainingArguments(
        output_dir=config.full_output_dir,
        save_total_limit=config.save_total_limit,
        save_strategy="epoch",
        load_best_model_at_end=True,
        save_only_model=True,
        metric_for_best_model='propaganda_f1',
        greater_is_better=True,
        logging_strategy='epoch',
        evaluation_strategy="epoch",
        learning_rate=5e-5,
        per_device_train_batch_size=config.train_batch_size,
        per_device_eval_batch_size=config.eval_batch_size,
        num_train_epochs=config.num_train_epochs,
        weight_decay=0.02,
        warmup_steps=num_warmup_steps,
        gradient_accumulation_steps=config.gradient_accumulation_steps,
        fp16=True,
        gradient_checkpointing=True,
        label_smoothing_factor=0,
        dataloader_num_workers=4,
        dataloader_pin_memory=True,
        optim="adamw_torch",
        remove_unused_columns=True,
        max_grad_norm=1.0,
    )

    trainer = PropagandaThresholdTrainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        data_collator=data_collator,  # Add the custom data collator
        callbacks=[
            EarlyStoppingCallback(
                early_stopping_patience=3,
                early_stopping_threshold=0.001
            ),
            MetricsCallback(
                output_dir=config.full_output_dir,
                id_to_label=config.id_to_label
            )
        ],
        compute_metrics=compute_metrics,
    )
    
    return trainer

In [3]:
def main():
    # Configuration
    config = TokenClassificationConfig()
    
    # Load data
    with open('/home/lgiordano/LUCA/checkthat_GITHUB/data/formatted/train.json', 'r', encoding='utf8') as f:
        dataset = json.load(f)

    texts = [item['text'] for item in dataset]
    annotations = [item['annotations'] for item in dataset]

    # Split data
    texts_train, texts_val, annotations_train, annotations_val = train_test_split(
        texts, annotations, test_size=0.2, random_state=42
    )

    # Initialize model and tokenizer
    tokenizer = AutoTokenizer.from_pretrained(config.model_name)
    model = setup_model(config.model_name, config.num_labels)

    # Create label_to_id mapping from id_to_label
    label_to_id = {v: k for k, v in config.id_to_label.items()}

    # Preprocess data
    train_inputs, train_labels = preprocess_data(
        texts_train, annotations_train, tokenizer, config, label_to_id
    )
    val_inputs, val_labels = preprocess_data(
        texts_val, annotations_val, tokenizer, config, label_to_id
    )

    # Create datasets with pad token ID
    train_dataset = TokenClassificationDataset(train_inputs, train_labels, tokenizer.pad_token_id)
    val_dataset = TokenClassificationDataset(val_inputs, val_labels, tokenizer.pad_token_id)

    # Setup and run training
    trainer = setup_training(config, train_dataset, val_dataset, model, tokenizer)
    trainer.train()

if __name__ == "__main__":
    main()

/opt/miniconda3/envs/checkthat/lib/python3.8/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/opt/miniconda3/envs/checkthat/lib/python3.8/site-packages/transformers/convert_slow_tokenizer.py:560: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Some weights of DebertaV2ForTokenClassification were not initialized from the model checkpoint at microsoft/mdeberta-v3-base and are newly initialized: ['classifier.bias', 'classif

Epoch,Training Loss,Validation Loss,Propaganda Precision,Propaganda Recall,Propaganda F1,Per Class Precision,Per Class Recall,Per Class F1,Support
0,1.584800,1.333735,0.000000,0.000000,0.000000,"[0.7229969723422623, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.9990462993423785, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.8388958181610177, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[601866, 8435, 2417, 564, 12872, 1922, 4844, 10996, 2722, 4196, 0, 1003, 4077, 1008, 3325, 34652, 2783, 12052, 1057, 28028, 49243, 6602, 13387, 24189]"
2,1.168200,1.265712,0.157456,0.011390,0.021244,"[0.7339846720879892, 0.07938360961942563, 0.0, 0.0, 0.0, 0.0005841121495327102, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.2604166666666667, 0.0, 0.0, 0.0, 0.3048543689320388, 0.24159248269040554, 0.0, 0.0877228119367114, 0.010025062656641603]","[0.979723061279421, 0.12092471843509188, 0.0, 0.0, 0.0, 0.0015608740894901144, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0007214590788410481, 0.0, 0.0, 0.0, 0.00560154131582703, 0.019840383404747884, 0.0, 0.032718308807051615, 0.00016536442184464012]","[0.839234947556889, 0.09584664536741214, 0.0, 0.0, 0.0, 0.0008500991782374609, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0014389317370783932, 0.0, 0.0, 0.0, 0.011000945941211504, 0.03666935650346238, 0.0, 0.04766050054406964, 0.0003253619651862697]","[601866, 8435, 2417, 564, 12872, 1922, 4844, 10996, 2722, 4196, 0, 1003, 4077, 1008, 3325, 34652, 2783, 12052, 1057, 28028, 49243, 6602, 13387, 24189]"
4,1.053300,1.378802,0.262915,0.017541,0.032888,"[0.7309002159016239, 0.19618304945618714, 0.0, 0.0, 0.012942519984773505, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.16666666666666666, 0.0, 0.0, 0.02309782608695652, 0.0, 0.03018867924528302, 0.0, 0.29155313351498635, 0.15851025548758546, 0.0, 0.0, 0.08174791914387634]","[0.9702674681739789, 0.11333728512151749, 0.0, 0.0, 0.0026413921690490987, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0002452783909737552, 0.0, 0.0, 0.0004905921736119127, 0.0, 0.0006637902422834385, 0.0, 0.0038176109604681034, 0.05367260321263936, 0.0, 0.0, 0.011368804001819008]","[0.8337434441167841, 0.14367297865945297, 0.0, 0.0, 0.004387379830956836, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0004898359049718343, 0.0, 0.0, 0.0009607776647451114, 0.0, 0.0012990176179264433, 0.0, 0.007536538122908962, 0.08019175629958887, 0.0, 0.0, 0.01996152869016078]","[601866, 8435, 2417, 564, 12872, 1922, 4844, 10996, 2722, 4196, 0, 1003, 4077, 1008, 3325, 34652, 2783, 12052, 1057, 28028, 49243, 6602, 13387, 24189]"
6,0.913300,1.478083,0.242587,0.033415,0.058739,"[0.7402003418812362, 0.10246863617968434, 0.0, 0.0, 0.029649595687331536, 0.0, 0.0, 0.0, 0.013842482100238664, 0.0, 0.0, 0.0, 0.010869565217391304, 0.0, 0.0, 0.09395532194480946, 0.0, 0.00938337801608579, 0.0, 0.2304147465437788, 0.18434025770917623, 0.0, 0.20601755119097367, 0.04867803311094638]","[0.9403289104219211, 0.15008891523414344, 0.0, 0.0, 0.001709136109384711, 0.0, 0.0, 0.0, 0.0106539309331374, 0.0, 0.0, 0.0, 0.0029433406916850625, 0.0, 0.0, 0.0041267459309707955, 0.0, 0.0011616329239960172, 0.0, 0.0035678607107178534, 0.11010702028714742, 0.0, 0.03682677224172705, 0.008144197775848527]","[0.8283483075599047, 0.12178932178932178, 0.0, 0.0, 0.003231967092698692, 0.0, 0.0, 0.0, 0.012040689225659124, 0.0, 0.0, 0.0, 0.004632310364794441, 0.0, 0.0, 0.007906230994637031, 0.0, 0.0020673360897814526, 0.0, 0.007026913077085236, 0.13786615134255492, 0.0, 0.06248415716096324, 0.013953817821221137]","[601866, 8435, 2417, 564, 12872, 1922, 4844, 10996, 2722, 4196, 0, 1003, 4077, 1008, 3325, 34652, 2783, 12052, 1057, 28028, 49243, 6602, 13387, 24189]"
8,0.776800,1.599403,0.172833,0.021435,0.038139,"[0.7359712267866161, 0.1018664563617245, 0.0, 0.0, 0.044167610419026046, 0.0, 0.0, 0.00898045430


Propaganda Detection Metrics:
Precision: 0.0000
Recall: 0.0000
F1: 0.0000

Per-class F1 scores (excluding 'O'):
Appeal_to_Authority: 0.0000 (support: 8435)
Appeal_to_Popularity: 0.0000 (support: 2417)
Appeal_to_Values: 0.0000 (support: 564)
Appeal_to_Fear-Prejudice: 0.0000 (support: 12872)
Flag_Waving: 0.0000 (support: 1922)
Causal_Oversimplification: 0.0000 (support: 4844)
False_Dilemma-No_Choice: 0.0000 (support: 10996)
Consequential_Oversimplification: 0.0000 (support: 2722)
Straw_Man: 0.0000 (support: 4196)
Whataboutism: 0.0000 (support: 1003)
Slogans: 0.0000 (support: 4077)
Appeal_to_Time: 0.0000 (support: 1008)
Conversation_Killer: 0.0000 (support: 3325)
Loaded_Language: 0.0000 (support: 34652)
Repetition: 0.0000 (support: 2783)
Exaggeration-Minimisation: 0.0000 (support: 12052)
Obfuscation-Vagueness-Confusion: 0.0000 (support: 1057)
Name_Calling-Labeling: 0.0000 (support: 28028)
Doubt: 0.0000 (support: 49243)
Guilt_by_Association: 0.0000 (support: 6602)
Appeal_to_Hypocrisy: 0.0

/opt/miniconda3/envs/checkthat/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/miniconda3/envs/checkthat/lib/python3.8/site-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(



Propaganda Detection Metrics:
Precision: 0.0000
Recall: 0.0000
F1: 0.0000

Per-class F1 scores (excluding 'O'):
Appeal_to_Authority: 0.0000 (support: 8435)
Appeal_to_Popularity: 0.0000 (support: 2417)
Appeal_to_Values: 0.0000 (support: 564)
Appeal_to_Fear-Prejudice: 0.0000 (support: 12872)
Flag_Waving: 0.0000 (support: 1922)
Causal_Oversimplification: 0.0000 (support: 4844)
False_Dilemma-No_Choice: 0.0000 (support: 10996)
Consequential_Oversimplification: 0.0000 (support: 2722)
Straw_Man: 0.0000 (support: 4196)
Whataboutism: 0.0000 (support: 1003)
Slogans: 0.0000 (support: 4077)
Appeal_to_Time: 0.0000 (support: 1008)
Conversation_Killer: 0.0000 (support: 3325)
Loaded_Language: 0.0000 (support: 34652)
Repetition: 0.0000 (support: 2783)
Exaggeration-Minimisation: 0.0000 (support: 12052)
Obfuscation-Vagueness-Confusion: 0.0000 (support: 1057)
Name_Calling-Labeling: 0.0000 (support: 28028)
Doubt: 0.0000 (support: 49243)
Guilt_by_Association: 0.0000 (support: 6602)
Appeal_to_Hypocrisy: 0.0

...


Propaganda Detection Metrics:
Precision: 0.1607
Recall: 0.0428
F1: 0.0676

Per-class F1 scores (excluding 'O'):
Appeal_to_Authority: 0.1098 (support: 8435)
Appeal_to_Popularity: 0.0000 (support: 2417)
Appeal_to_Values: 0.0028 (support: 564)
Appeal_to_Fear-Prejudice: 0.0188 (support: 12872)
Flag_Waving: 0.0062 (support: 1922)
Causal_Oversimplification: 0.0228 (support: 4844)
False_Dilemma-No_Choice: 0.0244 (support: 10996)
Consequential_Oversimplification: 0.0141 (support: 2722)
Straw_Man: 0.0182 (support: 4196)
Whataboutism: 0.0506 (support: 1003)
Slogans: 0.0012 (support: 4077)
Appeal_to_Time: 0.0000 (support: 1008)
Conversation_Killer: 0.0000 (support: 3325)
Loaded_Language: 0.0434 (support: 34652)
Repetition: 0.0056 (support: 2783)
Exaggeration-Minimisation: 0.0064 (support: 12052)
Obfuscation-Vagueness-Confusion: 0.0181 (support: 1057)
Name_Calling-Labeling: 0.0440 (support: 28028)
Doubt: 0.1129 (support: 49243)
Guilt_by_Association: 0.0000 (support: 6602)
Appeal_to_Hypocrisy: 0.0

/opt/miniconda3/envs/checkthat/lib/python3.8/site-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(



Propaganda Detection Metrics:
Precision: 0.1541
Recall: 0.0427
F1: 0.0668

Per-class F1 scores (excluding 'O'):
Appeal_to_Authority: 0.0944 (support: 8435)
Appeal_to_Popularity: 0.0000 (support: 2417)
Appeal_to_Values: 0.0025 (support: 564)
Appeal_to_Fear-Prejudice: 0.0265 (support: 12872)
Flag_Waving: 0.0092 (support: 1922)
Causal_Oversimplification: 0.0156 (support: 4844)
False_Dilemma-No_Choice: 0.0175 (support: 10996)
Consequential_Oversimplification: 0.0373 (support: 2722)
Straw_Man: 0.0315 (support: 4196)
Whataboutism: 0.0112 (support: 1003)
Slogans: 0.0065 (support: 4077)
Appeal_to_Time: 0.0016 (support: 1008)
Conversation_Killer: 0.0000 (support: 3325)
Loaded_Language: 0.0329 (support: 34652)
Repetition: 0.0000 (support: 2783)
Exaggeration-Minimisation: 0.0159 (support: 12052)
Obfuscation-Vagueness-Confusion: 0.0080 (support: 1057)
Name_Calling-Labeling: 0.0465 (support: 28028)
Doubt: 0.1079 (support: 49243)
Guilt_by_Association: 0.0003 (support: 6602)
Appeal_to_Hypocrisy: 0.0

/opt/miniconda3/envs/checkthat/lib/python3.8/site-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(



Propaganda Detection Metrics:
Precision: 0.1508
Recall: 0.0405
F1: 0.0639

Per-class F1 scores (excluding 'O'):
Appeal_to_Authority: 0.0953 (support: 8435)
Appeal_to_Popularity: 0.0000 (support: 2417)
Appeal_to_Values: 0.0032 (support: 564)
Appeal_to_Fear-Prejudice: 0.0210 (support: 12872)
Flag_Waving: 0.0037 (support: 1922)
Causal_Oversimplification: 0.0129 (support: 4844)
False_Dilemma-No_Choice: 0.0383 (support: 10996)
Consequential_Oversimplification: 0.0188 (support: 2722)
Straw_Man: 0.0403 (support: 4196)
Whataboutism: 0.0370 (support: 1003)
Slogans: 0.0029 (support: 4077)
Appeal_to_Time: 0.0020 (support: 1008)
Conversation_Killer: 0.0000 (support: 3325)
Loaded_Language: 0.0304 (support: 34652)
Repetition: 0.0000 (support: 2783)
Exaggeration-Minimisation: 0.0142 (support: 12052)
Obfuscation-Vagueness-Confusion: 0.0000 (support: 1057)
Name_Calling-Labeling: 0.0411 (support: 28028)
Doubt: 0.1018 (support: 49243)
Guilt_by_Association: 0.0005 (support: 6602)
Appeal_to_Hypocrisy: 0.0